In [ ]:
from OpenGL.GL import *
from OpenGL.GLUT import *
from OpenGL.GLU import *
from OpenGL.GLUT import GLUT_BITMAP_HELVETICA_18

import sys
import math

import sys
import math
print(sys.executable)
import pygame

pygame.init()
pygame.mixer.init()
alarm_sound = pygame.mixer.Sound(r"G:\cse423\CSE423-3D-Game\air-raid-siren-225303.mp3")

alarm_played = False


# Camera control
camera_angle = [0, 0]
camera_pos = [0, 5, 20]

# Flag positions
red_flag_pos = [-10, 0, -10]
blue_flag_pos = [10, 0, 10]
# player positions
player_pos = [0.0, 0.0, 0.0]
player_speed = 0.1
key_states = {b"w": False, b"s": False, b"a": False, b"d": False}
player_health = 5
# Direction player is facing: [dx, dy, dz]
player_direction = [0, 0, 1]  # Initially facing +Z (forward)

thunder_flash_timer = 0  # Duration in frames (0 = no flash)

player_has_flag = False
game_over = False
toggle_mode = False  # Starts off

player_wins = False

fireballs = []  # List of fireballs, each is a dict with position and velocity
fireball_speed = 1.0

tower_pos = [10.0, 0.0, 7.0]  # Near the blue flag
tower_fireballs = []
tower_fireball_speed = 0.1
tower_cooldown = 0  # Countdown until next fire
tower_alerted = False


def init():
    glClearColor(0.5, 0.8, 1.0, 1.0)  # Sky blue background
    glEnable(GL_DEPTH_TEST)
    glShadeModel(GL_SMOOTH)
    glutKeyboardFunc(key_down)
    glutKeyboardUpFunc(key_up)
    glutMouseFunc(mouse_click)

def draw_text(x, y, text, font=GLUT_BITMAP_HELVETICA_18):
    glColor3f(1, 1, 1)
    glMatrixMode(GL_PROJECTION)
    glPushMatrix()
    glLoadIdentity()
    gluOrtho2D(0, 800, 0, 600)
    glMatrixMode(GL_MODELVIEW)
    glPushMatrix()
    glLoadIdentity()
    glRasterPos2f(x, y)
    for ch in text:
        glutBitmapCharacter(font, ord(ch))
    glPopMatrix()
    glMatrixMode(GL_PROJECTION)
    glPopMatrix()
    glMatrixMode(GL_MODELVIEW)


def reshape(w, h):
    glViewport(0, 0, w, h)
    glMatrixMode(GL_PROJECTION)
    glLoadIdentity()
    gluPerspective(60.0, float(w)/float(h), 1.0, 100.0)
    glMatrixMode(GL_MODELVIEW)

def draw_flag(position, color):
    glPushMatrix()
    glTranslatef(*position)
    glColor3fv(color)
    glutSolidCube(1.0)
    glPopMatrix()

def draw_base(position, color):
    glPushMatrix()
    glTranslatef(*position)
    glScalef(4, 0.2, 4)
    glColor3fv(color)
    glutSolidCube(1.0)
    glPopMatrix()

# Assume this is your shared ground render function
def render_ground():
    glColor3f(0.3, 0.6, 0.3)  # Grass green
    glBegin(GL_QUADS)
    glVertex3f(-50, -0.1, -50)
    glVertex3f(-50, -0.1, 50)
    glVertex3f(50, -0.1, 50)
    glVertex3f(50, -0.1, -50)
    glEnd()

def reset_game():
    global player_pos, player_health, player_has_flag, player_wins
    global fireballs, tower_fireballs, tower_cooldown
    global tower_alerted, game_over

    tower_alerted = False
    game_over = False

    player_pos[:] = [0.0, 0.0, 0.0]
    player_health = 5
    player_has_flag = False
    player_wins = False
    fireballs.clear()
    tower_fireballs.clear()
    tower_cooldown = 0

    print("🔄 Game Reset!")


def check_player_hit():
    global tower_fireballs, player_pos, player_health, player_has_flag, game_over

    new_tower_fireballs = []
    for fb in tower_fireballs:
        dist = distance(fb["pos"], player_pos)
        if dist < 1.2:
            player_health -= 1
            if player_health <= 0:
                game_over = True
                print("💀 Game Over! Player has been eliminated.")
            continue
        new_tower_fireballs.append(fb)

    tower_fireballs[:] = new_tower_fireballs



def trigger_thunder_flash():
    global thunder_flash_timer
    thunder_flash_timer = 5  # Flash for 5 frames


def display():
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    glLoadIdentity()

    if game_over:
        draw_text(300, 300, "💀 GAME OVER", GLUT_BITMAP_HELVETICA_18)
        glutSwapBuffers()
        return
    if toggle_mode:
        eye = [player_pos[0], player_pos[1] + 1.0, player_pos[2]]
        look_distance = 10
        center = [player_pos[0] + player_direction[0] * look_distance,
                  player_pos[1] + player_direction[1] * look_distance,
                  player_pos[2] + player_direction[2] * look_distance]

        gluLookAt(eye[0], eye[1], eye[2],
                  center[0], center[1], center[2],
                  0, 1, 0)

        render_ground()

        # 🌳 Draw trees
        def draw_tree(x, z):
            glPushMatrix()
            glTranslatef(x, 0.5, z)
            glColor3f(0.55, 0.27, 0.07)
            glScalef(0.2, 1.0, 0.2)
            glutSolidCube(1.0)
            glPopMatrix()

            glPushMatrix()
            glTranslatef(x, 1.2, z)
            glColor3f(0.0, 0.5, 0.0)
            glutSolidSphere(0.5, 10, 10)
            glPopMatrix()

        for x in range(-45, 50, 15):
            draw_tree(x, -45)
            draw_tree(x, 45)
        for z in range(-30, 50, 15):
            draw_tree(-45, z)
            draw_tree(45, z)

        # 🚧 Draw boundary walls
        def draw_wall(x, z, x_scale, z_scale):
            glPushMatrix()
            glTranslatef(x, 1.0, z)
            glScalef(x_scale, 2.0, z_scale)
            glColor3f(0.6, 0.6, 0.6)
            glutSolidCube(1.0)
            glPopMatrix()

        draw_wall(0, -49, 100, 1)
        draw_wall(0, 49, 100, 1)
        draw_wall(-49, 0, 1, 100)
        draw_wall(49, 0, 1, 100)

        draw_base([-10, 0, -10], (1, 0, 0))
        draw_base([10, 0, 10], (0, 0, 1))

        if not player_has_flag:
            draw_flag(blue_flag_pos, (0, 0, 1))
        draw_flag(red_flag_pos, (1, 0, 0))

        for fb in fireballs:
            glPushMatrix()
            glTranslatef(*fb["pos"])
            glColor3f(1, 0.5, 0)
            glutSolidSphere(0.3, 10, 10)
            glPopMatrix()

        glPushMatrix()
        glTranslatef(*tower_pos)
        glColor3f(0.3, 0.3, 0.3)
        glScalef(1, 3, 1)
        glutSolidCube(1.0)
        glPopMatrix()

        for fb in tower_fireballs:
            glPushMatrix()
            glTranslatef(*fb["pos"])
            glColor3f(1.0, 0.1, 0.1)
            glutSolidSphere(0.3, 10, 10)
            glPopMatrix()

        draw_text(10, 570, f"Health: {player_health}", font=GLUT_BITMAP_HELVETICA_18)
        glutSwapBuffers()
        return

    update_player()
    check_flag_pickup()
    update_fireballs()
    update_tower()
    update_tower_fireballs()
    check_player_hit()

    gluLookAt(camera_pos[0], camera_pos[1], camera_pos[2],
              0, 0, 0,
              0, 1, 0)

    render_ground()

    # 🌳 Draw trees
    def draw_tree(x, z):
        glPushMatrix()
        glTranslatef(x, 0.5, z)
        glColor3f(0.55, 0.27, 0.07)
        glScalef(0.2, 1.0, 0.2)
        glutSolidCube(1.0)
        glPopMatrix()

        glPushMatrix()
        glTranslatef(x, 1.2, z)
        glColor3f(0.0, 0.5, 0.0)
        glutSolidSphere(0.5, 10, 10)
        glPopMatrix()

    for x in range(-45, 50, 15):
        draw_tree(x, -45)
        draw_tree(x, 45)
    for z in range(-30, 50, 15):
        draw_tree(-45, z)
        draw_tree(45, z)

    # 🚧 Draw boundary walls
    def draw_wall(x, z, x_scale, z_scale):
        glPushMatrix()
        glTranslatef(x, 1.0, z)
        glScalef(x_scale, 2.0, z_scale)
        glColor3f(0.6, 0.6, 0.6)
        glutSolidCube(1.0)
        glPopMatrix()

    draw_wall(0, -49, 100, 1)
    draw_wall(0, 49, 100, 1)
    draw_wall(-49, 0, 1, 100)
    draw_wall(49, 0, 1, 100)

    draw_base([-10, 0, -10], (1, 0, 0))
    draw_base([10, 0, 10], (0, 0, 1))

    if not player_has_flag:
        draw_flag(blue_flag_pos, (0, 0, 1))
    draw_flag(red_flag_pos, (1, 0, 0))

    glPushMatrix()
    glTranslatef(*player_pos)
    glPushMatrix()
    glScalef(1.2, 0.4, 1.5)
    glColor3f(0.4, 0.4, 0.4)
    glutSolidCube(1.0)
    glPopMatrix()

    glPushMatrix()
    glTranslatef(0, 0.4, 0)
    glScalef(0.6, 0.3, 0.6)
    glColor3f(0.7, 0.7, 0.0)
    glutSolidCube(1.0)
    glPopMatrix()

    glPushMatrix()
    glTranslatef(player_direction[0] * 0.6, 0.4, player_direction[2] * 0.6)
    glRotatef(math.degrees(math.atan2(player_direction[0], player_direction[2])), 0, 1, 0)
    glColor3f(0.2, 0.2, 0.2)
    glTranslatef(0, 0, 0.4)
    glScalef(0.1, 0.1, 0.8)
    glutSolidCube(1.0)
    glPopMatrix()

    glPopMatrix()

    if player_wins:
        draw_text(300, 300, "🏁 YOU WIN!", GLUT_BITMAP_HELVETICA_18)

    for fb in fireballs:
        glPushMatrix()
        glTranslatef(*fb["pos"])
        glColor3f(1, 0.5, 0)
        glutSolidSphere(0.3, 10, 10)
        glPopMatrix()

    glPushMatrix()
    glTranslatef(*tower_pos)
    glColor3f(0.3, 0.3, 0.3)
    glScalef(1, 3, 1)
    glutSolidCube(1.0)
    glPopMatrix()

    for fb in tower_fireballs:
        glPushMatrix()
        glTranslatef(*fb["pos"])
        glColor3f(1.0, 0.1, 0.1)
        glutSolidSphere(0.3, 10, 10)
        glPopMatrix()

    draw_text(10, 570, f"Health: {player_health}", font=GLUT_BITMAP_HELVETICA_18)
    global thunder_flash_timer

    if thunder_flash_timer > 0:
        glDisable(GL_DEPTH_TEST)
        glEnable(GL_BLEND)
        glBlendFunc(GL_SRC_ALPHA, GL_ONE_MINUS_SRC_ALPHA)
        glColor4f(1.0, 1.0, 1.0, 0.5)  # white semi-transparent

        glMatrixMode(GL_PROJECTION)
        glPushMatrix()
        glLoadIdentity()
        gluOrtho2D(0, 800, 0, 600)
        glMatrixMode(GL_MODELVIEW)
        glPushMatrix()
        glLoadIdentity()

        glBegin(GL_QUADS)
        glVertex2f(0, 0)
        glVertex2f(800, 0)
        glVertex2f(800, 600)
        glVertex2f(0, 600)
        glEnd()

        glPopMatrix()
        glMatrixMode(GL_PROJECTION)
        glPopMatrix()
        glMatrixMode(GL_MODELVIEW)
        glDisable(GL_BLEND)
        glEnable(GL_DEPTH_TEST)
        thunder_flash_timer -= 1
    global alarm_played
    if distance(player_pos, tower_pos) < 2.0:
        if not alarm_played:
            alarm_sound.play()
            alarm_played = True
    else:
        alarm_played = False
    glutSwapBuffers()



def key_down(key, x, y):
    global toggle_mode
    if key in key_states:
        key_states[key] = True
    elif key == b' ':  # Spacebar
        shoot_fireball()
    elif key == b't':  # 't' for toggle
        toggle_mode = not toggle_mode
        print("🟢 Toggle Mode:", "ON" if toggle_mode else "OFF")

def key_up(key, x, y):
    if key in key_states:
        key_states[key] = False

def mouse_click(button, state, x, y):
    if button == GLUT_LEFT_BUTTON and state == GLUT_DOWN:
        if game_over or player_wins:
            reset_game()


def update_player():
    global player_direction

    if key_states[b"w"]:
        player_pos[2] -= player_speed
        player_direction = [0, 0, -1]
    if key_states[b"s"]:
        player_pos[2] += player_speed
        player_direction = [0, 0, 1]
    if key_states[b"a"]:
        player_pos[0] -= player_speed
        player_direction = [-1, 0, 0]
    if key_states[b"d"]:
        player_pos[0] += player_speed
        player_direction = [1, 0, 0]

def distance(p1, p2):
    return math.sqrt(sum([(p1[i] - p2[i])**2 for i in range(3)]))

def check_flag_pickup():
    global player_has_flag, player_wins, tower_alerted

    if not player_has_flag and distance(player_pos, blue_flag_pos) < 1.5:
        player_has_flag = True
        trigger_thunder_flash()  # ⚡ Flash on flag capture
        print("🚩 Blue Flag Captured!")

    elif player_has_flag and distance(player_pos, red_flag_pos) < 1.5:
        player_wins = True
        player_has_flag = False
        tower_alerted = False  # 🧠 Stop the tower!
        print("🏁 Player Wins!")



def shoot_fireball():
    fireball = {
        "pos": player_pos[:],       # copy current position
        "vel": player_direction[:]  # copy current facing direction
    }
    fireballs.append(fireball)


def update_fireballs():
    global fireballs, tower_alerted

    new_fireballs = []
    for fb in fireballs:
        # Update position
        fb["pos"] = [fb["pos"][i] + fb["vel"][i] * fireball_speed for i in range(3)]

        # Check for tower collision
        if distance(fb["pos"], tower_pos) < 1.5:
            tower_alerted = True
            print("🚨 Tower alerted by player's fireball!")
            continue  # Remove this fireball after hitting

        # Keep fireball in scene bounds
        if -50 < fb["pos"][0] < 50 and -50 < fb["pos"][2] < 50:
            new_fireballs.append(fb)

    fireballs = new_fireballs


def update_tower():
    global tower_cooldown

    if (player_has_flag or tower_alerted) and not player_wins:
        if tower_cooldown <= 0:
            shoot_tower_fireball()
            tower_cooldown = 60
        else:
            tower_cooldown -= 1




def shoot_tower_fireball():
    dir_vec = [player_pos[i] - tower_pos[i] for i in range(3)]
    magnitude = math.sqrt(sum([x**2 for x in dir_vec]))
    if magnitude == 0:
        return
    velocity = [x / magnitude for x in dir_vec]
    fireball = {
        "pos": tower_pos[:],
        "vel": velocity
    }
    tower_fireballs.append(fireball)

def update_tower_fireballs():
    global tower_fireballs
    new_list = []
    for fb in tower_fireballs:
        fb["pos"] = [fb["pos"][i] + fb["vel"][i] * tower_fireball_speed for i in range(3)]
        if -50 < fb["pos"][0] < 50 and -50 < fb["pos"][2] < 50:
            new_list.append(fb)
    tower_fireballs = new_list

check_player_hit()


def main():
    glutInit(sys.argv)
    glutInitDisplayMode(GLUT_DOUBLE | GLUT_RGB | GLUT_DEPTH)
    glutInitWindowSize(800, 600)
    glutCreateWindow(b"Capture the Flag 3D Environment")
    init()
    glutDisplayFunc(display)
    glutReshapeFunc(reshape)
    glutIdleFunc(glutPostRedisplay)
    glutMainLoop()

if __name__ == '__main__':
    main()


C:\Users\LENOVO\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe
pygame 2.6.1 (SDL 2.28.4, Python 3.12.10)
Hello from the pygame community. https://www.pygame.org/contribute.html
🚨 Tower alerted by player's fireball!
🚩 Blue Flag Captured!
🏁 Player Wins!
🚩 Blue Flag Captured!
🏁 Player Wins!
🔄 Game Reset!
🚩 Blue Flag Captured!
🏁 Player Wins!
🔄 Game Reset!
🚨 Tower alerted by player's fireball!
🚨 Tower alerted by player's fireball!
💀 Game Over! Player has been eliminated.
🔄 Game Reset!
🟢 Toggle Mode: ON
🟢 Toggle Mode: OFF
🟢 Toggle Mode: ON
🟢 Toggle Mode: OFF
🚩 Blue Flag Captured!
🏁 Player Wins!
🚩 Blue Flag Captured!
🔄 Game Reset!
🚩 Blue Flag Captured!
🏁 Player Wins!
💀 Game Over! Player has been eliminated.
🔄 Game Reset!
🚩 Blue Flag Captured!
🏁 Player Wins!
🔄 Game Reset!
🚨 Tower alerted by player's fireball!
🚩 Blue Flag Captured!
🏁 Player Wins!
🚩 Blue Flag Captured!
🏁 Player Wins!
